In [1]:
import copy
import torch
import deepsnap
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn

from sklearn.metrics import f1_score
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from torch_sparse import SparseTensor, matmul

import pickle
import networkx as nx

In [2]:
class HeteroGNNConv(pyg_nn.MessagePassing):
    def __init__(self, in_channels_src, in_channels_dst, out_channels):
        super(HeteroGNNConv, self).__init__(aggr="mean")

        self.in_channels_src = in_channels_src
        self.in_channels_dst = in_channels_dst
        self.out_channels = out_channels

        self.lin_dst = None
        self.lin_src = None

        self.lin_update = None

        self.lin_dst = nn.Linear(in_channels_dst, out_channels)
        self.lin_src = nn.Linear(in_channels_src, out_channels)
        self.lin_update = nn.Linear(2 * out_channels, out_channels)

    def forward(
        self,
        node_feature_src,
        node_feature_dst,
        edge_index,
        size=None,
        res_n_id=None,
        ):

        return self.propagate(edge_index, node_feature_src=node_feature_src, 
                    node_feature_dst=node_feature_dst, size=size, res_n_id=res_n_id)

    def message_and_aggregate(self, edge_index, node_feature_src):

        out = matmul(edge_index, node_feature_src, reduce='mean')

        return out

    def update(self, aggr_out, node_feature_dst, res_n_id):

        dst_out = self.lin_dst(node_feature_dst)
        aggr_out = self.lin_src(aggr_out)
        aggr_out = torch.cat([dst_out, aggr_out], -1)
        aggr_out = self.lin_update(aggr_out)

        return aggr_out

In [3]:
class HeteroGNNWrapperConv(deepsnap.hetero_gnn.HeteroConv):
    def __init__(self, convs, args, aggr="mean"):
        """
        Initializes the HeteroGNNWrapperConv instance.

        :param convs: Dictionary of convolution layers for each message type.
        :param args: Arguments dictionary containing hyperparameters like hidden_size and attn_size.
        :param aggr: Aggregation method, defaults to 'mean'.
        """
        
        super(HeteroGNNWrapperConv, self).__init__(convs, None)
        self.aggr = aggr

        # Map the index and message type
        self.mapping = {}

        # A numpy array that stores the final attention probability
        self.alpha = None

        self.attn_proj = None

        if self.aggr == "attn":

            self.attn_proj = nn.Sequential(
                nn.Linear(args['hidden_size'], args['attn_size']),
                nn.Tanh(),
                nn.Linear(args['attn_size'], 1, bias=False)
            )
    
    def reset_parameters(self):
        super(HeteroGNNWrapperConv, self).reset_parameters()
        if self.aggr == "attn":
            for layer in self.attn_proj.children():
                layer.reset_parameters()
    
    def forward(self, node_features, edge_indices):
        """
        Forward pass of the model.

        :param node_features: Dictionary of node features for each node type.
        :param edge_indices: Dictionary of edge indices for each message type.
        :return: Aggregated node embeddings for each node type.
        """
        
        message_type_emb = {}
        for message_key, message_type in edge_indices.items():
            src_type, edge_type, dst_type = message_key
            node_feature_src = node_features[src_type]
            node_feature_dst = node_features[dst_type]
            edge_index = edge_indices[message_key]
            message_type_emb[message_key] = (
                self.convs[message_key](
                    node_feature_src,
                    node_feature_dst,
                    edge_index,
                )
            )
            
        
        node_emb = {dst: [] for _, _, dst in message_type_emb.keys()}
        mapping = {}        
        
        for (src, edge_type, dst), item in message_type_emb.items():
            mapping[len(node_emb[dst])] = (src, edge_type, dst)
            node_emb[dst].append(item)
        self.mapping = mapping
        
        for node_type, embs in node_emb.items():
            if len(embs) == 1:
                node_emb[node_type] = embs[0]
            else:
                node_emb[node_type] = self.aggregate(embs)
                
        return node_emb
    
    def aggregate(self, xs):
        """
        Aggregates node embeddings using the specified aggregation method.

        :param xs: List of node embeddings to aggregate.
        :return: Aggregated node embeddings as a torch.Tensor.
        """

        if self.aggr == "mean":
            xs = torch.stack(xs)
            out = torch.mean(xs, dim=0)
            return out

        elif self.aggr == "attn":
            xs = torch.stack(xs, dim=0)
            s = self.attn_proj(xs).squeeze(-1)
            s = torch.mean(s, dim=-1)
            self.alpha = torch.softmax(s, dim=0).detach()
            out = self.alpha.reshape(-1, 1, 1) * xs
            out = torch.sum(out, dim=0)
            return out

In [4]:
def generate_convs(hetero_graph, conv, hidden_size, first_layer=False):
    
    """
    Generates convolutional layers for each message type in a heterogeneous graph.

    :param hetero_graph: The heterogeneous graph for which convolutions are to be created.
    :param conv: The convolutional layer class or constructor.
    :param hidden_size: The number of features in the hidden layer.
    :param first_layer: Boolean indicating if this is the first layer in the network.
    
    :return: A dictionary of convolutional layers, keyed by message type.
    """

    convs = {}
    
    # Extracting all types of messages/edges in the heterogeneous graph.
    all_messages_types = hetero_graph.message_types
    for message_type in all_messages_types:
        
        # Determine the input feature size for source and destination nodes.
        # If it's the first layer, use the feature size of the nodes.
        # Otherwise, use the hidden size, since from there on the size of embeddings
        # is the same for all nodes.
        if first_layer:
            in_channels_src = hetero_graph.num_node_features(message_type[0])
            in_channels_dst = hetero_graph.num_node_features(message_type[2])
        else:
            in_channels_src = hidden_size
            in_channels_dst = hidden_size
        out_channels = hidden_size
        
        # Create a convolutional layer for this message type and add it to the dictionary.
        convs[message_type] = conv(in_channels_src, in_channels_dst, out_channels)
    
    return convs

In [5]:
class HeteroGNN(torch.nn.Module):
    def __init__(self, hetero_graph, args, num_layers, aggr="mean"):
        super(HeteroGNN, self).__init__()

        self.aggr = aggr
        self.hidden_size = args['hidden_size']

        self.bns1 = nn.ModuleDict()
        self.bns2 = nn.ModuleDict()
        self.relus1 = nn.ModuleDict()
        self.relus2 = nn.ModuleDict()
        self.post_mps = nn.ModuleDict()
        self.fc = nn.ModuleDict()
        
        # Initialize the graph convolutional layers
        self.convs1 = HeteroGNNWrapperConv(
            generate_convs(hetero_graph, HeteroGNNConv, self.hidden_size, first_layer=True), 
            args, self.aggr)
        self.convs2 = HeteroGNNWrapperConv(
            generate_convs(hetero_graph, HeteroGNNConv, self.hidden_size, first_layer=False), 
            args, self.aggr)

        # Initialize batch normalization, ReLU, and fully connected layers for each node type
        all_node_types = hetero_graph.node_types
        for node_type in all_node_types:
            
            self.bns1[node_type] = nn.BatchNorm1d(self.hidden_size, eps=1.0)
            self.bns2[node_type] = nn.BatchNorm1d(self.hidden_size, eps=1.0)
            
            self.relus1[node_type] = nn.LeakyReLU()
            self.relus2[node_type] = nn.LeakyReLU()
            self.fc[node_type] = nn.Linear(self.hidden_size, 1)
            
    def forward(self, node_feature, edge_index):
        """
        Forward pass of the model.

        :param node_feature: Dictionary of node features for each node type.
        :param edge_index: Dictionary of edge indices for each message type.
        :return: The output embeddings for each node type after passing through the model.
        """
        x = node_feature

        # Apply graph convolutional, batch normalization, and ReLU layers
        x = self.convs1(x, edge_index)
        x = forward_op(x, self.bns1)
        x = forward_op(x, self.relus1)

        x = self.convs2(x, edge_index)
        x = forward_op(x, self.bns2)
        x = forward_op(x, self.relus2)
        
        x = forward_op(x, self.fc)

        return x

    def loss(self, preds, y, indices):
        """
        Computes the loss for the model.

        :param preds: Predictions made by the model.
        :param y: Ground truth target values.
        :param indices: Indices of nodes for which loss should be calculated.
        
        :return: The computed loss value.
        """
        
        loss = 0
        loss_func = torch.nn.MSELoss() 
             
        mask = y['event'][indices['event'], 0] != -1
        non_zero_idx = torch.masked_select(indices['event'], mask)
                
        loss += loss_func(preds['event'][non_zero_idx], y['event'][non_zero_idx])

        return loss

In [6]:
def train(model, optimizer, hetero_graph, train_idx):
    """
    Trains the model on the given heterogeneous graph using the specified indices.

    :param model: The graph neural network model to train.
    :param optimizer: The optimizer used for training the model.
    :param hetero_graph: The heterogeneous graph data.
    :param train_idx: Indices for training nodes.

    :return: The training loss as a float.
    """

    model.train() # Set the model to training mode
    optimizer.zero_grad() # Zero out any existing gradients 
    
    # Compute predictions using the model
    # TODO: Use only train_idx instead of edge_index
    # TODO: Train only on events not on concepts
    
    preds = model(hetero_graph.node_feature, hetero_graph.edge_index)

    # Compute the loss using model's loss function
    loss = model.loss(preds, hetero_graph.node_target, train_idx)

    loss.backward() # Backward pass: compute gradient of the loss
    optimizer.step() # Perform a single optimization step, updates parameters
    
    return loss.item() 

def test(model, graph, indices, best_model, best_tvt_scores):
    """
    Tests the model on given indices and updates the best model based on validation loss.

    :param model: The trained graph neural network model.
    :param graph: The heterogeneous graph data.
    :param indices: List of indices for training, validation, and testing nodes.
    :param best_model: The current best model based on validation loss.
    :param best_val: The current best validation loss.
    
    :return: A tuple containing the list of losses for each dataset, the best model, and the best validation loss.
    """
    
    model.eval() # Set the model to evaluation mode
    tvt_scores = []
    
    # Evaluate the model on each set of indices
    for index in indices:
        preds = model(graph.node_feature, graph.edge_index)
        
        idx = index['event']
        
                     
        # mask = y['event'][indices['event'], 0] != -1
        # non_zero_idx = torch.masked_select(indices['event'], mask)
        #preds['event'][non_zero_idx], y['event'][non_zero_idx]
        
        # non_zero_targets = torch.masked_select(graph.node_target['event'][indices['event']], mask)
        # non_zero_truth = torch.masked_select(graph.node_target['event'][indices['event']], mask)
        
        mask = graph.node_target['event'][idx, 0] != -1
        non_zero_idx = torch.masked_select(idx, mask)
        
        
        L1 = torch.sum(torch.abs(preds['event'][non_zero_idx] - graph.node_target['event'][non_zero_idx])) / non_zero_idx.shape[0]
        
        tvt_scores.append(L1)
    
    # Update the best model and validation loss if the current model performs better
    if tvt_scores[1] < best_tvt_scores[1]:
        best_tvt_scores = tvt_scores
        # torch.to_pickle(model, 'best_model.pkl')
        # model.to_pickle('best_model.pkl')
        # best_model = copy.deepcopy(model)
        torch.save(model.state_dict(), './best_model.pkl')
    
    return tvt_scores, best_tvt_scores, best_model

In [7]:
args = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'hidden_size': 48,
    'epochs': 500,
    'weight_decay': 0.0002930387278908051,
    'lr': 0.05091434725288385,
    'attn_size': 32,
}

In [8]:
"""
This cell creates a small heterogeneous graph, primarily for testing purposes.
"""

S_node_feature = {
    "event": torch.tensor([
                [1, 1, 1],   # event 0
                [2, 2, 2]    # event 1
    ], dtype=torch.float32),
    "concept": torch.tensor([
                [2, 2, 2],   # concept 0
                [3, 3, 3]    # concept 1
    ], dtype=torch.float32)
}

# S_node_label = {
#     "event": torch.tensor([0, 1], dtype=torch.long), # Class 0, Class 1
#     "concept": torch.tensor([0, 1], dtype=torch.long)  # Class 0, Class 1
# }

S_node_targets = {
    "event": torch.tensor([[50], [2000]], dtype=torch.float32),
    # "concept": torch.tensor([[0], [0]], dtype=torch.float32)
}

S_edge_index = {
    ("event", "similar", "event"): torch.tensor([[0,1],[1,0]], dtype=torch.int64),
    ("event", "related", "concept"): torch.tensor([[0,1],[0,1]], dtype=torch.int64),
    ("concept", "related", "event"): torch.tensor([[0,1],[0,1]], dtype=torch.int64)
}

# Testing
hetero_graph = HeteroGraph(
    node_feature=S_node_feature,
    node_target=S_node_targets,
    edge_index=S_edge_index
)

train_idx = {"event": torch.tensor([0, 1]).to(args['device']), "concept": torch.tensor([0, 1]).to(args['device'])}
val_idx = {"event": torch.tensor([0, 1]).to(args['device']), "concept": torch.tensor([0, 1]).to(args['device'])}
test_idx = {"event": torch.tensor([0, 1]).to(args['device']), "concept": torch.tensor([0, 1]).to(args['device'])}

In [9]:
with open("./1_concepts_similar_llm.pkl", "rb") as f:
    G = pickle.load(f)
    # Convert to directed graph for compatibility with Deepsnap
    # G = G.to_directed()
   
        
hetero_graph = HeteroGraph(G, netlib=nx, directed=True)

In [10]:
"""
This code block ensures that all tensors in a heterogeneous graph are transferred to the same 
computational device, as specified in the 'args' dictionary.
"""

for message_type in hetero_graph.message_types:
    print("TYPE", message_type)
    print("\t Feature", hetero_graph.num_node_features(message_type[0]))
    print("\t Feature", hetero_graph.num_node_features(message_type[2]))

# Send node features to device
for key in hetero_graph.node_feature:
    hetero_graph.node_feature[key] = hetero_graph.node_feature[key].to(args['device'])

# for key in hetero_graph.node_label:
#     hetero_graph.node_label[key] = hetero_graph.node_label[key].to(args['device'])

# Create a torch.SparseTensor from edge_index and send it to device
for key in hetero_graph.edge_index:
    print("KEY", key, type(key))
    print("KEY NUMS", key, hetero_graph.num_nodes(key[0]), hetero_graph.num_nodes(key[2]))
    
    edge_index = hetero_graph.edge_index[key]

    print("MAX EDGES", edge_index[0].max(), edge_index[1].max(), hetero_graph.num_nodes(key[0]), hetero_graph.num_nodes(key[2]))
    adj = SparseTensor(row=edge_index[0].long(), col=edge_index[1].long(), sparse_sizes=(hetero_graph.num_nodes(key[0]), hetero_graph.num_nodes(key[2])))
    hetero_graph.edge_index[key] = adj.t().to(args['device'])
    
# Send node targets to device
for key in hetero_graph.node_target:
    hetero_graph.node_target[key] = hetero_graph.node_target[key].to(args['device'])

TYPE ('event', 'similar', 'event')
	 Feature 769
	 Feature 769
TYPE ('event', 'related', 'concept')
	 Feature 769
	 Feature 1
TYPE ('concept', 'related', 'event')
	 Feature 1
	 Feature 769
KEY ('event', 'similar', 'event') <class 'tuple'>
KEY NUMS ('event', 'similar', 'event') 8487 8487
MAX EDGES tensor(8283) tensor(8283) 8487 8487
KEY ('event', 'related', 'concept') <class 'tuple'>
KEY NUMS ('event', 'related', 'concept') 8487 8729
MAX EDGES tensor(8265) tensor(8728) 8487 8729
KEY ('concept', 'related', 'event') <class 'tuple'>
KEY NUMS ('concept', 'related', 'event') 8729 8487
MAX EDGES tensor(8728) tensor(8265) 8729 8487


In [11]:
"""
This code block creates a basic split of a graph's nodes into training, validation, and testing sets. 
It uses predefined ratios to divide 'event' and 'concept' nodes in the heterogeneous graph for a simple 
dataset split, mainly for testing purposes.
"""

nEvents = hetero_graph.num_nodes("event")
nConcepts = hetero_graph.num_nodes("concept")

s1 = 0.7
s2 = 0.8

train_idx = {   "event": torch.tensor(range(0, int(nEvents * s1))).to(args["device"]), 
                "concept": torch.tensor(range(0, int(nConcepts * s1))).to(args["device"])
            }
val_idx = {   "event": torch.tensor(range(int(nEvents * s1), int(nEvents * s2))).to(args["device"]), 
                "concept": torch.tensor(range(int(nConcepts * s1), int(nConcepts * s2))).to(args["device"])
            }
test_idx = {   "event": torch.tensor(range(int(nEvents * s2), nEvents)).to(args["device"]), 
                "concept": torch.tensor(range(int(nConcepts * s2), nConcepts)).to(args["device"])
            }

print(train_idx["event"].shape)
print(test_idx["event"].shape)
print(val_idx["event"].shape)


# TODO: Add node labels to the nodes and try to make the deepsnap split work even for regression!

# dataset = deepsnap.dataset.GraphDataset([hetero_graph], task='node')

# dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.4, 0.3, 0.3])
# datasets = {'train': dataset_train, 'val': dataset_val, 'test': dataset_test}

# datasets

torch.Size([5940])
torch.Size([1698])
torch.Size([849])


In [12]:
"""
Creates a HeteroGNN model from the previously constructed hetero graph and trains it.
"""

best_model = None
best_tvt_scores = (float("inf"),float("inf"),float("inf"))

model = HeteroGNN(hetero_graph, args, num_layers=2, aggr="mean").to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

for epoch in range(args['epochs']):
    # Train
    loss = train(model, optimizer, hetero_graph, train_idx)
    # Test for the accuracy of the model
    cur_tvt_scores, best_tvt_scores, best_model = test(model, hetero_graph, [train_idx, val_idx, test_idx], best_model, best_tvt_scores)
    print(f"Epoch {epoch} Loss {loss:.4f} Current Train,Val,Test Scores {[score.item() for score in cur_tvt_scores]}")

print("Best Train,Val,Test Scores", [score.item() for score in best_tvt_scores])

Epoch 0 Loss 2814.4502 Current Train,Val,Test Scores [951.3971557617188, 938.8259887695312, 493.22601318359375]
Epoch 1 Loss 2432.2859 Current Train,Val,Test Scores [7026.90869140625, 7020.7880859375, 2313.303955078125]
Epoch 2 Loss 2083.7654 Current Train,Val,Test Scores [4856.0947265625, 4852.65625, 1663.0482177734375]
Epoch 3 Loss 1744.9337 Current Train,Val,Test Scores [2084.755615234375, 2085.794189453125, 781.117919921875]
Epoch 4 Loss 1673.4425 Current Train,Val,Test Scores [639.6581420898438, 640.46875, 314.1707458496094]
Epoch 5 Loss 1692.3171 Current Train,Val,Test Scores [382.3239440917969, 382.997314453125, 253.8238525390625]
Epoch 6 Loss 1702.0675 Current Train,Val,Test Scores [234.04428100585938, 231.4447021484375, 203.26171875]
Epoch 7 Loss 1628.2104 Current Train,Val,Test Scores [50.489505767822266, 63.394161224365234, 112.52481842041016]
Epoch 8 Loss 1516.0288 Current Train,Val,Test Scores [210.1905517578125, 222.51959228515625, 108.86294555664062]
Epoch 9 Loss 1480.35

In [13]:
model = HeteroGNN(hetero_graph, args, num_layers=2, aggr="mean").to(args['device'])
model.load_state_dict(torch.load('./best_model.pkl'))
preds = model(hetero_graph.node_feature, hetero_graph.edge_index)
# mask = preds['event'] > 0
# preds['event'][preds['event'] > 0].shape

# print(preds['event'][0], hetero_graph.node_target['event'][0]) 


#print(hetero_graph.node_feature['event'])

# Assuming val_idx is the dictionary containing the validation indices for 'event' node type

# TODO: Best validation score doesn't match the best validation score from the training loop
# best_val = torch.sum(torch.abs(preds['event'][val_idx['event']] - hetero_graph.node_target['event'][val_idx['event']])) / val_idx['event'].shape[0]
cur_tvt_scores, best_tvt_scores, best_model = test(model, hetero_graph, [train_idx, val_idx, test_idx], best_model, best_tvt_scores)


# Print the validation loss
print(f"Best Validation Score: {cur_tvt_scores[1].item()}")

# for i in range(3000):
#     if hetero_graph.node_target['event'][i] != -1: # concepts have node target -1
#         print(preds['event'][i], hetero_graph.node_target['event'][i])
        
    
for i in range(1000):    
    if hetero_graph.node_target['event'][test_idx['event']][i] != -1:
        print(preds['event'][test_idx['event']][i], hetero_graph.node_target['event'][test_idx['event']][i])


# for i in range(1000):
#     # Extract the target value and check if it is not equal to -1
#     target = hetero_graph.node_target['event'][test_idx['event']][i].item()
#     if target != -1:
#         # Extract the prediction value
#         pred = preds['event'][test_idx['event']][i].item()
#         print(f"Prediction: {pred:.4f}, Target: {target:.4f}")


Best Validation Score: 28.15311050415039
tensor([7.2314], device='cuda:0', grad_fn=<SelectBackward0>) tensor([3.], device='cuda:0')
tensor([6.3395], device='cuda:0', grad_fn=<SelectBackward0>) tensor([5.], device='cuda:0')
tensor([88.9189], device='cuda:0', grad_fn=<SelectBackward0>) tensor([6.], device='cuda:0')
tensor([12.7852], device='cuda:0', grad_fn=<SelectBackward0>) tensor([94.], device='cuda:0')
tensor([171.6684], device='cuda:0', grad_fn=<SelectBackward0>) tensor([48.], device='cuda:0')
tensor([7.8729], device='cuda:0', grad_fn=<SelectBackward0>) tensor([4.], device='cuda:0')
tensor([125.8122], device='cuda:0', grad_fn=<SelectBackward0>) tensor([111.], device='cuda:0')
tensor([12.8855], device='cuda:0', grad_fn=<SelectBackward0>) tensor([7.], device='cuda:0')
tensor([11.3711], device='cuda:0', grad_fn=<SelectBackward0>) tensor([1.], device='cuda:0')
tensor([1.9716], device='cuda:0', grad_fn=<SelectBackward0>) tensor([1.], device='cuda:0')
tensor([5.1449], device='cuda:0', gra

In [14]:
import pickle

# Assuming you have already executed the model prediction code
# preds = model(hetero_graph.node_feature, hetero_graph.edge_index)

# Extract predictions and node IDs for the 'event' node type
event_predictions = preds['event'][test_idx['event']].detach().cpu().numpy()
event_node_ids = test_idx['event'].cpu().numpy()

# Combine predictions and node IDs into a dictionary
predictions_with_ids = {
    'node_ids': event_node_ids,
    'predictions': event_predictions
}

# Save the dictionary to a pickle file
with open('predictions_with_ids.pkl', 'wb') as f:
    pickle.dump(predictions_with_ids, f)

print("Predictions and node IDs saved to 'predictions_with_ids.pkl'")

Predictions and node IDs saved to 'predictions_with_ids.pkl'


### Hyperparameter Tuning

In [ ]:
import wandb
import optuna

def objective(trial):
    # Initialize wandb run
    wandb.init(project="V2_MLG_PredEvents_GNN+LMM", config={
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True),
        "hidden_size": trial.suggest_int("hidden_size", 16, 128),
        "attn_size": 32,  # Fixed value
        "epochs": trial.suggest_int("epochs", 150, 300),
        "num_layers": 2,  # Fixed value
    })

    # Use wandb config
    config = wandb.config

    # Initialize the model with the new hyperparameters
    model = HeteroGNN(hetero_graph, {
        'hidden_size': config.hidden_size,
        'attn_size': config.attn_size,
        'device': args['device']
    }, num_layers=config.num_layers, aggr="mean").to(args['device'])
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

    # Initialize best scores with infinity
    best_tvt_scores = (float("inf"), float("inf"), float("inf"))

    # Training loop
    for epoch in range(config.epochs):
        train_loss = train(model, optimizer, hetero_graph, train_idx)
        cur_tvt_scores, best_tvt_scores, _ = test(model, hetero_graph, [train_idx, val_idx, test_idx], None, best_tvt_scores)

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_score": cur_tvt_scores[1],
            "best_val_score": best_tvt_scores[1],
        })

        # Update the best validation score
        if cur_tvt_scores[1] < best_tvt_scores[1]:
            best_tvt_scores = (cur_tvt_scores[0], cur_tvt_scores[1], cur_tvt_scores[2])

    # Finish wandb run
    wandb.finish()

    # The objective value is the best validation score
    return best_tvt_scores[1]

# Create a study object and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)

# Print the best hyperparameters
print("Best trial:")
trial = study.best_trial

print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")